# Model 03

Try a really shallow autoencoder, one compression to 1/8 original input volume.


## Imports and Constants, etc.

In [ ]:
import datetime
import importlib
import keras
from keras.layers import (Dense, SimpleRNN, Input, Conv1D, 
                          LSTM, GRU, AveragePooling3D, Conv3D, 
                          UpSampling3D, BatchNormalization)
from keras.models import Model
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import projd
import random
import re
import scipy
import shutil
import sys
from sklearn.model_selection import train_test_split
import uuid

import matplotlib.pyplot as plt # data viz
import seaborn as sns # data viz

import imageio # display animated volumes
from IPython.display import Image # display animated volumes

from IPython.display import SVG # visualize model
from keras.utils.vis_utils import model_to_dot # visualize model

# for importing local code
src_dir = str(Path(projd.cwd_token_dir('notebooks')) / 'src') # $PROJECT_ROOT/src
if src_dir not in sys.path:
    sys.path.append(src_dir)

import util
importlib.reload(util)
import preprocessing
importlib.reload(preprocessing)
import datagen
importlib.reload(datagen)

SEED = 0
EPOCHS = 100
BATCH_SIZE = 4
PATCH_SHAPE = (32, 32, 32)

MODEL_NAME = 'model_03'

DATA_DIR = Path('/data2').expanduser()
NORMAL_SCANS_DIR = DATA_DIR / 'uvmmc/nifti_normals'
PROJECT_DATA_DIR = DATA_DIR / 'uvm_deep_learning_project'
PP_IMG_DIR = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed' # preprocessed scans dir
PP_MD_PATH = PROJECT_DATA_DIR / 'uvmmc' / 'preprocessed_metadata.pkl'

MODELS_DIR = PROJECT_DATA_DIR / 'models'
LOG_DIR = PROJECT_DATA_DIR / 'log'
TENSORBOARD_LOG_DIR = PROJECT_DATA_DIR / 'tensorboard' / MODEL_NAME
TMP_DIR = DATA_DIR / 'tmp'

for d in [DATA_DIR, NORMAL_SCANS_DIR, PROJECT_DATA_DIR, PP_IMG_DIR, MODELS_DIR, LOG_DIR, 
          TENSORBOARD_LOG_DIR, TMP_DIR, PP_MD_PATH.parent]:
    if not d.exists():
        d.mkdir(parents=True)
        
%matplotlib inline
sns.set()


## Data Generation

In [ ]:
train_gen, val_gen = datagen.get_nifti_datagens(preprocessed_metadata_path=PP_MD_PATH,
                                                batch_size=BATCH_SIZE, crop_shape=PATCH_SHAPE)


## Build Model


In [ ]:
def build_model(input_shape):
    '''
    Very simple 3D convolutional autoencoder.
    1 poolings reduce input from shape to shape/2, which in 3d is 1/8th the size of the original shape,
    a very respectable compression factor.
    '''
    n_a = 32

    x_input = Input(shape=input_shape) # shape:  (32, 32, 32, 1) == 32768
    x = x_input

    x = Conv3D(n_a, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # shape: (32, 32, 32, 32) = 1048576
    x = AveragePooling3D(padding='same')(x) # shape: (16, 16, 16, 32) = 131072

    x = Conv3D(n_a * 2, kernel_size=(3, 3, 3), padding='same', activation='relu')(x) # shape: (16, 16, 16, 64) = 262144
    x = UpSampling3D()(x) # shape (32, 32, 32, 64) = 2097152

    y = Conv3D(1, kernel_size=(3, 3, 3), padding='same', activation='sigmoid')(x) # shape: (32, 32, 32, 1) = 32768
    
    model = Model(inputs=x_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
   

In [ ]:
 
model = build_model(PATCH_SHAPE + (1,))
print(model.summary())
SVG(model_to_dot(model).create(prog='dot', format='svg'))

## Train and Evaluate Model

- Add callbacks to save model every 20 epochs and to log performance stats every epoch, so we have the results saved somewhere for charting.


In [ ]:
def get_model_path(models_dir, model_name, epoch):
    '''
    Paths match the template that the keras checkpoint callback uses to save models.
    '''
    model_path = models_dir  /  (model_name + f'_{epoch:02d}.h5')
    return model_path


def train_model_epoch(train_gen, val_gen, epoch, epochs=EPOCHS, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, 
                model_name=MODEL_NAME, log_dir=LOG_DIR,
                tensorboard_log_dir=TENSORBOARD_LOG_DIR, max_queue_size=10):
    
    path = get_model_path(models_dir, model_name, epoch)
    model = keras.models.load_model(path)
    return train_model(model=model, train_gen=train_gen, val_gen=val_gen, epochs=epochs, 
                       initial_epoch=epoch, batch_size=batch_size, models_dir=models_dir, 
                       model_name=model_name, log_dir=log_dir, tensorboard_log_dir=tensorboard_log_dir,
                       max_queue_size=10)


def train_model(model, train_gen, val_gen, epochs=EPOCHS, initial_epoch=0, batch_size=BATCH_SIZE, models_dir=MODELS_DIR, 
                model_name=MODEL_NAME, log_dir=LOG_DIR,
                tensorboard_log_dir=TENSORBOARD_LOG_DIR, max_queue_size=10):
    # Saving model
    model_path = models_dir  /  (model_name +'_{epoch:02d}.h5')
    print('model path:', model_path)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(
        str(model_path), monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, 
        mode='auto', period=1)
    
    # Stop when validation loss stops improving
    early_cb = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
    
    # Save logs for each run to logfile
    log_path = log_dir / (model_name + '_' + datetime.datetime.now().isoformat() + '_log.csv')
    print('log path:', log_path)
    log_cb = keras.callbacks.CSVLogger(str(log_path), separator=',', append=False)
    
    # Enable Tensorboard
    print('tensorboard log dir:', tensorboard_log_dir)
    tensorboard_cb = keras.callbacks.TensorBoard(log_dir=str(tensorboard_log_dir), 
                                                 histogram_freq=0, write_graph=True, write_images=True)
    
    # Fit Model
    history = model.fit_generator(train_gen, epochs=epochs, initial_epoch=initial_epoch, validation_data=val_gen, 
                        callbacks=[checkpoint_cb, log_cb, tensorboard_cb], max_queue_size=max_queue_size)
    return history, log_path

In [ ]:
history, log_path = train_model(model, train_gen, val_gen, epochs=40)
# history, log_path = train_model_epoch(train_gen, val_gen, epoch=100, epochs=200)

## Visualize Training Progress

In [ ]:
# read metrics from the log file
# log_path = LOG_DIR / (model_name + '_2018-04-26T17:29:02.902740_log.csv')
log_path = sorted(LOG_DIR.glob(f'{MODEL_NAME}*_log.csv'), reverse=True)[0]
print('most recent log_path:', log_path)

metrics = pd.read_csv(log_path)

In [ ]:
print(pd.concat([metrics[::10], metrics[-1:]])) # every 10th metric and the last one

In [ ]:
# Plot Training and Validation Accuracy 
axes = plt.gca()
axes.set_ylim([0.0,1.0]) # Show results on 0..1 range
plt.plot(metrics["acc"])
plt.plot(metrics["val_acc"])
plt.legend(['Training Accuracy', "Validation Accuracy"])
plt.show()

# Plot Training and Validation Loss
plt.plot(metrics["loss"])
plt.plot(metrics["val_loss"])
plt.legend(['Training Loss', "Validation Loss"])
plt.show()



## Show Effect of Autoencoder Training

Use models from different training epochs to encode images.


In [ ]:

def visualize_by_epochs(models_dir, model_name, epochs, train_gen, val_gen):
    kind_gens = (('train', train_gen), ('val', val_gen))
    kind_batches = [(kind, gen[0]) for kind, gen in kind_gens]
    for epoch in epochs:
            print('Epoch {}:'.format(epoch))
            path = get_model_path(models_dir, model_name, epoch)
            model = keras.models.load_model(path)
            for kind, (batch_x, batch_y) in kind_batches:
                print(f'{kind} data set')
                batch_pred = model.predict_on_batch(batch_x)
                for i in range(len(batch_x)):
                    print(f'predicted vs truth for image {i}')
                    display(util.animate_crop(np.squeeze(batch_pred[i])))
                    display(util.animate_crop(np.squeeze(batch_y[i])))
                        

In [ ]:
visualize_by_epochs(MODELS_DIR, MODEL_NAME, [1, 10, 40], train_gen, val_gen)